# Calculate Search Metrics with Dynamic Optimizer

This notebook loads the models that were built in the previous notebook and test how much the queries improve when using a pipeline with the predicted "neuralness" value.

Work-in-progress plan:

1. Get the models
2. Get the queries
3. Get the ratings file
4. Get the predicted "neuralness" scores for each query from each model
5. Set up the query pipelines
6. Run the queries (two runs per query; one per prediction)
7. Calculate search metrics
8. Compare search metrics with the best metrics

In [136]:
import pickle
import pandas as pd
import requests
import json
import uuid

In [137]:
# load the two models
with open('regression_model.pkl', 'rb') as fid:
    regr = pickle.load(fid)

with open('random_forest_model.pkl', 'rb') as fid:
    random_forest = pickle.load(fid)

In [138]:
# Import the ratings generated in the previous notebook
df_ratings = pd.read_csv('ratings.csv', sep="\t", names=['query', 'docid', 'rating', 'idx'])#, index=False)
df_ratings.head(5)

,query,docid,rating,idx
0,$30 roblox gift card not digital,B07RX6FBFR,3,0
1,$30 roblox gift card not digital,B09194H44R,0,0
2,$30 roblox gift card not digital,B08R5N6W6B,2,0
3,$30 roblox gift card not digital,B07Y693ND1,0,0
4,$30 roblox gift card not digital,B07RZ75JW3,2,0


In [139]:
df_test_data = pd.read_csv('dynamic_optimizer_test_data.csv')

In [140]:
df_test_data.head(5)

,query_string,ndcg,neuralness,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9
0,7 days without a pun makes one weak,0.664066,1.0,8,35,1,0,13,0.261906,2.187382,0.816212,0.714292
1,american flag 5x8 outdoor made in usa,0.595807,0.3,7,37,1,0,8,0.913549,4.974668,0.755891,0.723251
2,andis t edger,0.308602,1.0,3,13,0,0,2,0.086820,0.166090,0.578003,0.519416
3,backpacks lunchbox combo,0.084726,1.0,3,24,0,0,6,0.721674,1.747816,0.731043,0.656633
4,best ride on cars mercedes 3-in-1 push car in ...,0.426893,0.9,10,51,1,1,1,0.784587,0.784587,0.868892,0.631813


In [141]:
feature_columns = [f'f_{i}' for i in range(1, 10)]    
def get_linear_model_prediction(row):
    df_row = pd.DataFrame([row[feature_columns]], columns=feature_columns)
    return regr.predict(df_row)[0]

def get_random_forest_prediction(row):
    df_row = pd.DataFrame([row[feature_columns]], columns=feature_columns)
    return random_forest.predict(df_row)[0]

In [142]:
one_row = df_test_data[feature_columns].head(1)

In [143]:
one_row

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9
0,8,35,1,0,13,0.261906,2.187382,0.816212,0.714292


In [144]:
df_test_data['linear_model'] = df_test_data.apply(get_linear_model_prediction, axis=1)
df_test_data['random_forest'] = df_test_data.apply(get_random_forest_prediction, axis=1)

In [145]:
df_test_data.head(5)

,query_string,ndcg,neuralness,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,linear_model,random_forest
0,7 days without a pun makes one weak,0.664066,1.0,8,35,1,0,13,0.261906,2.187382,0.816212,0.714292,0.801477,0.435
1,american flag 5x8 outdoor made in usa,0.595807,0.3,7,37,1,0,8,0.913549,4.974668,0.755891,0.723251,0.741743,0.516
2,andis t edger,0.308602,1.0,3,13,0,0,2,0.086820,0.166090,0.578003,0.519416,0.735459,0.669
3,backpacks lunchbox combo,0.084726,1.0,3,24,0,0,6,0.721674,1.747816,0.731043,0.656633,0.780726,0.774
4,best ride on cars mercedes 3-in-1 push car in ...,0.426893,0.9,10,51,1,1,1,0.784587,0.784587,0.868892,0.631813,0.685039,0.609


In [146]:
# Get model_id
# We are assuming that the installation has only one model. Change this if you have more models 
# and need to pick a specific one

headers = {
    'Content-Type': 'application/json'
}

def get_model_id():
    url = "http://localhost:9200/_plugins/_ml/models/_search"
   
    payload = {
      "query": {
        "match_all": {}
      },
      "size": 1
    }
    
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

    return response.json()['hits']['hits'][0]['_source']['model_id']

model_id = get_model_id()

In [147]:
def create_search_pipeline(neuralness):
    neuralness = round(neuralness, 2)
    keywordness = 1 - neuralness

    pipeline_name = uuid.uuid4().hex[:8]


    payload = {
      "request_processors": [
        {
          "neural_query_enricher" : {
            "description": "one of many search pipelines for experimentation",
            "default_model_id": model_id,
            "neural_field_default_id": {
               "title_embeddings": model_id
            }
          }
        }
      ],
      "phase_results_processors": [
        {
          "normalization-processor": {
            "normalization": {
              "technique": "l2"
            },
            "combination": {
              "technique": "arithmetic_mean",
              "parameters": {
                "weights": [
                  keywordness,
                  neuralness
                ]
              }
            }
          }
        }
      ]    
    }

    url = "http://localhost:9200/_search/pipeline/" + pipeline_name
    
    response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
    #print(payload)
    
    return pipeline_name

In [148]:
models = ['linear_model', 'random_forest']
df_relevance = pd.DataFrame()

# iterate over all query strings and send a hybrid search query to OpenSearch with the set pipeline
for query in df_test_data.itertuples():
    for model in models:
        neuralness = df_test_data.loc[df_test_data['query_string'] == query[1], model].iloc[0]
        pipeline_name = ""
        pipeline_name = create_search_pipeline(neuralness)
       # print(pipeline_name)
        # Set pipeline 
        url = "http://localhost:9200/ecommerce/_search?search_pipeline=" + pipeline_name

    
        payload = {
          "_source": {
            "excludes": [
              "title_embedding"
            ]
          },
          "query": {
            "hybrid": {
              "queries": [
                {
                  "multi_match" : {
                      "type":       "best_fields",
                      "fields":     [
                        "product_id^100",
                        "product_bullet_point^3",
                        "product_color^2",
                        "product_brand^5",
                        "product_description",
                        "product_title^10"
                      ],
                      "operator":   "and",
                      "query":      query[1]
                    }
                },
                {
                  "neural": {
                    "title_embedding": {
                      "query_text": query[1],
                      "k": 100
                    }
                  }
                }
              ]
            }
          },
          "size": 100
        }
    
        response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
        #print(response)
        # store results per pipeline_id
        position = 0
        for hit in response['hits']['hits']:
            # create a new row for the DataFrame and append it
            row = { 'query_id' : str(query[0]), 'query_string': query[1], 'product_id' : hit["_id"], 'position' : str(position), 'relevance' : hit["_score"], 'run': model }
    
            new_row_df = pd.DataFrame([row])
            df_relevance = pd.concat([df_relevance, new_row_df], ignore_index=True)
            #print("%(id)s %(title)s: %(name)s" % hit["_source"])
            position += 1
    
    # work with two for loops:
    # 1) one to iterate over the list of queries and have a query id instead of a query
    # 2) another one to iterate over the result sets to have the position of the result in the result set 
    
    # DataFrame with columns:
    # query_id: the id of the query as the trec_eval tool needs a numeric id rather than a query string as an identifier
    # product_id: the id of the product in the hit list
    # position: the position of the product in the result set
    # relevance: relevance as given by the search engine
    # run: the name of the query pipeline

In [130]:
df_relevance

,query_id,query_string,product_id,position,relevance,run
0,0,7 days without a pun makes one weak,B07PCLLG2H,0,0.193235,linear_model
1,0,7 days without a pun makes one weak,B07PY9SL52,1,0.190547,linear_model
2,0,7 days without a pun makes one weak,B083Z9M4TR,2,0.186959,linear_model
3,0,7 days without a pun makes one weak,B07PGBYSYD,3,0.184747,linear_model
4,0,7 days without a pun makes one weak,B07RHGP8T8,4,0.181853,linear_model
...,...,...,...,...,...,...
995,4,best ride on cars mercedes 3-in-1 push car in ...,B07X8Z6PJ1,95,0.057144,random_forest
996,4,best ride on cars mercedes 3-in-1 push car in ...,B00N9YQ88C,96,0.057130,random_forest
997,4,best ride on cars mercedes 3-in-1 push car in ...,B00BJQKQH6,97,0.057104,random_forest
998,4,best ride on cars mercedes 3-in-1 push car in ...,B07DN2C3XL,98,0.057091,random_forest
